# PyTorch 基础 : 神经网络包nn和优化器optim

torch.nn是专门为神经网络设计的模块化接口。

nn构建于 Autograd之上，用来定义和运行神经网络。

这里主要介绍一些常用类

**约定：torch.nn 为了方便使用设置别名为nn，本章除nn外还有其他命名约定**

In [1]:
# 引入相关的包
import torch

# 引入torch.nn并指定别名
import torch.nn as nn

#打印一下版本
torch.__version__

'1.6.0'

除nn别名还引用nn.functional，包含神经网络使用的一些常用函数。

这些函数特点是，不具有可学习的参数(如ReLU，pool，DropOut等)，这些函数可放在构造函数中，也可以不放，建议不放。

一般情况下**将nn.functional 设置为大写的F**

In [2]:
import torch.nn.functional as F

## 定义一个网络

PyTorch已经准备好现成的网络模型，只要继承nn.Module实现forward方法，PyTorch会根据autograd自动实现backward函数，

forward函数可用任何tensor支持的函数，还可以用if、for、print、log等Python语法，写法和标准Python一致。

In [7]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()

#         self.conv1 = nn.Conv2d(1, 6, 3)
        
#         self.fc1 = nn.Linear(1350,10)
    
#     def forward(self,x):
#         print(x.size())
        
#         x = self.conv1(x)
#         x = F.relu(x)
#         print(x.size())
        
#         x = F.max_pool2d(x,(2,2))
#         x = F.relu(x)
#         print(x.size())
        
#         x = x.view(x.size()[0],-1)
#         print(x.size())
        
#         x = self.fc1(x)
#         return x
    
# net = Net()
# print(net)

In [33]:
class Net(nn.Module):
    def __init__(self):

        # nn.Module子类函数必须在构造函数中执行父类构造函数
        super(Net, self).__init__()

        # 卷积层 '1'表示输入图片单通道，'6'表示输出通道数，'3'表示卷积核为3*3
        self.conv1 = nn.Conv2d(1, 6, 3)

        #线性层，输入1350个特征，输出10个特征 
        self.fc1 = nn.Linear(1350, 10)

    #正向传播
    def forward(self, x):
        print(x.size())
        # 结果[1, 1, 32, 32]

        # 卷积 -> 激活 -> 池化
        # 根据卷积尺寸计算公式结果是30，具体计算公式后面第二章第四节 卷积神经网络 有详细介绍。
        x = self.conv1(x)
        x = F.relu(x)
        print(x.size())  
        # 结果[1, 6, 30, 30]
        
        
        # 使用池化层，计算结果15
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(x)
        print(x.size())  
        # 结果[1, 6, 15, 15]
        
        
        # 把[1, 6, 15, 15]压扁，变为 [1, 1350] reshape‘-1’表示自适应
        x = x.view(x.size()[0], -1)
        # fc1层的的输入1350
        print(x.size())  
        
        # 结果1350
        x = self.fc1(x)
        
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


网络可学习参数 通过net.parameters()返回

In [34]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[ 0.1404,  0.1430, -0.1102],
          [ 0.1459, -0.3215,  0.1739],
          [-0.2614,  0.1917,  0.2980]]],


        [[[ 0.3172, -0.2400, -0.2382],
          [-0.0769,  0.2274, -0.1209],
          [ 0.1064, -0.2572,  0.3016]]],


        [[[-0.2319,  0.1844, -0.0064],
          [-0.0077,  0.0490,  0.0914],
          [-0.1213,  0.0078,  0.0014]]],


        [[[ 0.1305, -0.0403,  0.2858],
          [-0.1147, -0.0619,  0.0637],
          [ 0.1482,  0.2279, -0.0653]]],


        [[[ 0.2717,  0.0172, -0.1815],
          [ 0.3134,  0.3091,  0.2698],
          [-0.0592, -0.2693, -0.0740]]],


        [[[-0.0225, -0.0798, -0.2458],
          [-0.1924, -0.2072, -0.0565],
          [ 0.0949,  0.3195, -0.1648]]]], requires_grad=True)
Parameter containing:
tensor([-0.1863,  0.0619,  0.0005, -0.1337, -0.0773,  0.1795],
       requires_grad=True)
Parameter containing:
tensor([[-0.0216,  0.0082,  0.0058,  ...,  0.0035, -0.0171, -0.0166],
        [-0.0243,  0.0244,  0

net.named_parameters可同时返回可学习的参数及名称

In [35]:
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


forward函数的输入和输出都是Tensor

In [36]:
# forward的输入是32
input = torch.randn(1, 1, 32, 32)
out = net(input)
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [37]:
input.size()

torch.Size([1, 1, 32, 32])

反向传播前，要将所有参数的梯度清零

In [38]:
net.zero_grad() 

# PyTorch自动实现反向传播，只要调用这个函数即可
out.backward(torch.ones(1,10)) 

In [40]:
out

tensor([[-0.2480,  0.2751, -0.0508,  0.9324, -0.9408, -0.1811,  0.5845, -0.3075,
          0.0055,  0.4399]], grad_fn=<AddmmBackward>)

**注意**:torch.nn只支持mini-batches，不支持一次只输入一个样本，一次必须是一个batch。

就算输入一个样本，也会对样本进行分批，所有输入都会增加一个维度。

对比刚才的input，nn定义为3维，但人工创建时多增加一个维度变为4维，最前面的1即为batch-size。

## 损失函数

nn中PyTorch预制了常用损失函数，用MSELoss计算均方误差

In [39]:
y = torch.arange(0,10).view(1,10).float()
y.size()

torch.Size([1, 10])

In [41]:
criterion = nn.MSELoss()
loss = criterion(out, y)

#loss是scalar，直接用item获取python类型的数值
print(loss.item()) 

28.0238037109375


## 优化器

反向传播计算所有参数的梯度后，要用优化方法更新网络的权重和参数，如随机梯度下降法(SGD)的更新策略如下：

weight = weight - learning_rate * gradient

torch.optim实现大多数的优化方法，如RMSProp、Adam、SGD等，用SGD做个简单样例

In [43]:
import torch.optim
for i in range(10):
    # 调用时打印forword函数的大小
    out = net(input) 
    criterion = nn.MSELoss()
    loss = criterion(out, y)
    print(loss,"\n")

    # 新建一个优化器，SGD调整的参数和学习率
    optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)

    # 梯度清零(与net.zero_grad()效果一样)
    optimizer.zero_grad() 
    loss.backward()

    #更新参数
    optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0478, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0216, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0081, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0034, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0013, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0006, grad_fn=<MseLossBackward>) 

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
tensor(0.0002, 

神经网络数据的一个完整传播就已经通过PyTorch实现。

下一章介绍PyTorch提供的数据加载和处理工具，这些工具可以方便的处理所需要的数据。

大家可能对神经网络模型里面的一些参数的计算方式还有疑惑，这部分会在第二章 第四节 卷积神经网络有详细介绍，
并且在第三章 第二节 MNIST数据集手写数字识别的实践代码中有详细的注释说明。